In [52]:
from ffiec_data_connect import methods, credentials, ffiec_connection
import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.connector.pandas_tools import write_pandas
import time
from zeep.exceptions import Fault
import datetime
from pathlib import Path
import numpy as np 
from snowflake.connector.errors import ProgrammingError
#creds = credentials.WebserviceCredentials(username="your-username", password="your-token")
#creds = credentials.WebserviceCredentials(username="whills12", password="vGzHZX7EAsPCwfXuV6pI")
#conn = ffiec_connection.FFIECConnection()

In [62]:
def format_callreports(p,fp, y, q):
    parts= q.split('_')
    replacement_dict = {'0': f'{y}{parts[0]}{parts[1]}', '': f'{y}{parts[0]}{parts[1]}'} 
    datefill = f'{y}-{parts[0]}-{parts[1]}'
    yeardf = pd.DataFrame()
    yearflag = 0
    ct=0
    plist = list(p.glob(f'**\\{y}\\{q}\\*'))
    if not plist:
        print(f'the following folder didnt exist/has nothing {y}:{q}')
        
    for pl in plist:
        if ct == 0:
            dfinitial = pd.read_csv(pl, sep='\t', on_bad_lines='warn',engine="python") 
            dfinitial.drop(dfinitial.columns[-1],axis=1,inplace=True)
            dfinitial['IDRSSD'] = dfinitial['IDRSSD'].astype(int)
            ct += 1
            continue
        else: 
            dftomerge = pd.read_csv(pl, sep='\t', on_bad_lines='warn', engine="python")
            dftomerge.drop(dftomerge.columns[-1],axis=1,inplace=True)
            dftomerge.drop(0,inplace=True)
        print(str(pl))
        dftomerge['IDRSSD'] = pd.to_numeric(dftomerge['IDRSSD'], errors = 'coerce').dropna()

        dfinitial = pd.merge(dfinitial, dftomerge, on=['IDRSSD'], how='left', suffixes=('', '_y'))
        dfinitial.drop(dfinitial.filter(regex='_y$').columns, axis=1, inplace=True)

    if yearflag==0:
        yeardf = dfinitial
        yearflag+=1
    else:
        yeardf = pd.concat([yeardf, dfinitial], axis=0, ignore_index=True)
        print(len(yeardf))        
    yeardf.drop(list(yeardf.filter(regex = 'TEXT').columns),axis = 1, inplace = True)
    try:
        yeardf['RCON9999']=pd.to_datetime(yeardf['RCON9999'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    except ValueError:
        yeardf['RCON9999'] = yeardf['RCON9999'].replace(replacement_dict)
        yeardf['RCON9999']=pd.to_datetime(yeardf['RCON9999'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    yeardf.replace('CONF', None, inplace=True)
    #yeardf['RCON9999'] = yeardf['RCON9999'].fillna(datefill)
    yeardf.dropna(subset=['IDRSSD'], how='any', inplace=True)
    yeardf['RCON9999']=yeardf['RCON9999'].replace(np.NaN, f'{y}-{parts[0]}-{parts[1]}')
    return yeardf

In [3]:
def initialize_snowflake_connection(username,password,account,role,warehouse,database,schema):
    connection_parameters = {
        "user": username,
        "password": password,
        "account": account,
        "role": role,
        "warehouse": warehouse,
        "database": database,
        "schema": schema }
    return snowpark.session.Session.builder.configs(connection_parameters).create()

In [4]:
def initialize_ffeic_connection(username, password):
    creds = credentials.WebserviceCredentials(username=username, password=password)
    conn = ffiec_connection.FFIECConnection()
    return creds, conn

In [5]:
def print_row_count(session: snowpark.Session, t):
    row_count = session.sql(f"SELECT COUNT(*) AS row_count FROM {t};").collect()
    df = pd.DataFrame(row_count)
    return print(f'{df["ROW_COUNT"][0]} values inserted into table {t}')

In [6]:
def grab_table_name(session: snowpark.Session):
    tables = session.sql('''SELECT TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA = 'PUBLIC';''').collect()
    table_names = pd.DataFrame(tables)
    table_list = table_names['TABLE_NAME'].tolist()
    table_list.remove('SCHEDULE_METADATA')
    return table_list

In [7]:
def grab_table_schema(session: snowpark.Session, table_name):
    df_schema = session.sql(f"DESCRIBE TABLE {table_name};").collect()
    schema_cols = pd.DataFrame(df_schema)
    return pd.DataFrame(columns=schema_cols['name'].tolist()), schema_cols['name'].tolist()

In [8]:
def grab_id_data(reporting_period, passed_list, creds, conn):
    skipper = 0
    limit_caller = 0
    error_list = []
    give_it_back = 0
    try:
        if not passed_list:
            limit_caller += 1
            give_it_back = 2499
            raise ValueError('The passed ID RSSD list is empty, calling the webservice')
        else:
            id_rssd_list = passed_list 
            print('Using the passed id list')
    except ValueError as e:
        print(e)
        filers = methods.collect_filers_on_reporting_period(
        session=conn,
        creds=creds,
        reporting_period=reporting_period,
        output_type="pandas")
        id_rssd_list = sorted(filers['id_rssd'].tolist())
    print('Processing period: ', reporting_period, f"list length is {len(id_rssd_list)}")
    print("The time was : ", time.ctime())
    for id_rssd in id_rssd_list:
        if limit_caller == 2500:
            break
        else:
            try:
                df = methods.collect_data(
                    session=conn,
                    creds=creds,
                    rssd_id=id_rssd,
                    reporting_period=reporting_period,
                    series="call",
                    output_type='pandas'
                )
                result_df = pd.DataFrame(columns=df['mdrm'].unique())
                for index, row in df.iterrows():
                    mdrm = row['mdrm']
                    data_type = row['data_type']
                    if data_type == 'str':
                        value = row['str_data']
                    elif data_type == 'bool':
                        value = row['bool_data']
                    elif data_type == 'float':
                        value = row['float_data']
                    elif data_type == 'int':
                        value = row['int_data']
                    else:
                        value = None
                    result_df.at[0, mdrm] = value
                result_df.insert(0, 'IDRSSD',id_rssd)
                result_df['RCON9999']=pd.to_datetime(result_df['RCON9999'], format='%Y%m%d.%f').dt.strftime('%Y-%m-%d')
                date_column = result_df.pop('RCON9999') 
                result_df.insert(1, 'RCON9999', date_column) 
                if skipper == 0:
                    final_df = result_df
                    skipper+=1
                else:
                    final_df = pd.concat([final_df,result_df]).reset_index(drop=True)
                limit_caller+=1
            except Fault as e:
                print(f'The IDRSSD: {id_rssd} is not available for reporting period {reporting_period}: printing fault statement {e}')
                error_list.append(id_rssd)
                limit_caller+=1
                continue
    if give_it_back == 0:
        id_rssd_partial = id_rssd_list[limit_caller:]
    else: 
        id_rssd_partial = id_rssd_list[give_it_back:]
    print("Records to still be processed: ", len(id_rssd_partial))
    return final_df, error_list, id_rssd_partial

In [9]:
def main(session: snowpark.Session, creds, conn, is_df,reporting_period,period_string):
    print('Grabbing Data')
    if is_df:
        passed_list = []
    else: 
        passed_df=pd.read_csv(f'C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\Uploading Files\\{period_string}_partial_id_list.csv')
        passed_list = passed_df['0'].tolist()
    #passed_list = [] 
    #passed_df=pd.read_csv(f'C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\Uploading Files\\{period_string}_partial_id_list.csv')
    #passed_list = passed_df['0'].tolist()
    #pd.read_csv(f'C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\Uploading Files\\{reporting_period}_error_list.csv')
    final_df, error_list, id_rssd_partial = grab_id_data(reporting_period, passed_list, creds, conn)
    print("The last data call was : ", time.ctime())
    if error_list:
        print('saving errors')
        e_df = pd.DataFrame(error_list)
        e_df.to_csv(f'C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\Uploading Files\\{period_string}_error_list.csv',index=False)
    if id_rssd_partial: 
        print('saving partial list')
        partial_id_df = pd.DataFrame(id_rssd_partial)
        partial_id_df.to_csv(f'C:\\Users\\whill\\OneDrive\\Desktop\\Fall2022\\ISYE6740\\call-report-database\\Uploading Files\\{period_string}_partial_id_list.csv',index=False)
    table_list = grab_table_name(session)
    print("Started writing data to snowflake @: ", time.ctime())
    for t in table_list:
        concat_df, col_list = grab_table_schema(session, t)
        write_df = pd.concat([concat_df,final_df.filter(items=col_list)])
        session.write_pandas(write_df,t)
        print_row_count(session, t)

In [17]:
if __name__ == "__main__":
    warehouse = "COMPUTE_WH"
    session = initialize_snowflake_connection(username= "",password = "",account= "",role = "",
                                    warehouse = "",database = "",schema = "")
    creds, conn = initialize_ffeic_connection(username='',password="")
    reporting_period = '03/31/2020'
    period_string = '03_31_2020'
    main(session, creds, conn, False, reporting_period, period_string)
    session.sql(f"ALTER WAREHOUSE {warehouse} SUSPEND;")
    session.close()

Grabbing Data
Using the passed id list
Processing period:  03/31/2020 list length is 166
The time was :  Fri Dec 15 15:32:16 2023
Records to still be processed:  0
The last data call was :  Fri Dec 15 15:33:16 2023
Started writing data to snowflake @:  Fri Dec 15 15:33:18 2023
73426 values inserted into table SCHEDULE_RCCI
73426 values inserted into table SCHEDULE_RCD
73426 values inserted into table SCHEDULE_RCEII
73426 values inserted into table SCHEDULE_RCF
73426 values inserted into table SCHEDULE_RCG
73426 values inserted into table SCHEDULE_RCS
73426 values inserted into table SCHEDULE_RCT
73426 values inserted into table SCHEDULE_RCN
73426 values inserted into table SCHEDULE_RCV
73426 values inserted into table SCHEDULE_RIC
73426 values inserted into table SCHEDULE_LEO
73426 values inserted into table SCHEDULE_RCH
73426 values inserted into table SCHEDULE_RCI
73426 values inserted into table SCHEDULE_RCM
73426 values inserted into table SCHEDULE_RIBII
73426 values inserted into 

In [12]:
def main(session: snowpark.Session, df, fp,y,q):
    table_list = grab_table_name(session)
    for t in table_list:
        try:
            concat_df, col_list = grab_table_schema(session, t)
            write_df = pd.concat([concat_df,df.filter(items=col_list)])
            write_df.to_csv(f'{fp}\\{t}\\{y}_CallReports_{q}.csv',index=False)
            session.write_pandas(write_df,t)
            print_row_count(session, t)
        except ProgrammingError as pe:
            print(f"Error in table {t}: {pe}")
            write_df.to_csv(f'{fp}\\Failed_Uploads\\{y}_{t}_error_upload_report_{q}.csv',index=False)

In [66]:
if __name__ == "__main__":
    warehouse = "COMPUTE_WH"
    session = initialize_snowflake_connection(username= "",password = "",account= "",role = "",
                                    warehouse = "",database = "",schema = "")
    p = Path('C:\\Users\\whill\\call-report-database\\Bulk_Files')
    fp = 'C:\\Users\\whill\\call-report-database\\Bulk_Files'
    years = [2013+ x for x in range(0,7)]
    quarters = ['03_31', '06_30', '09_30','12_31']
    for y in years:
        for q in quarters:
            df = format_callreports(p, fp, y, q)
            main(session, df,fp, y, q)

C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule CI 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule ENT 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule NARR 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RC 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RCA 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RCB 03312013(1 of 2).txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RCB 03312013(2 of 2).txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RCCI 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RCCII 03312013.txt
C:\Users\whill\call-report-database\Bulk_Files\2013\03_31\FFIEC CDR Call Schedule RCD 03312

C:\Users\whill\anaconda3\lib\site-packages\snowflake\snowpark\session.py:1891: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(


457297 values inserted into table SCHEDULE_RCE


C:\Users\whill\anaconda3\lib\site-packages\snowflake\snowpark\session.py:1891: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(


457297 values inserted into table SCHEDULE_RCEI


...

621401 values inserted into table SCHEDULE_RIE


C:\Users\whill\anaconda3\lib\site-packages\snowflake\snowpark\session.py:1891: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(


621401 values inserted into table SCHEDULE_RCV


C:\Users\whill\anaconda3\lib\site-packages\snowflake\snowpark\session.py:1891: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(


621401 values inserted into table SCHEDULE_RCE
